# 스타벅스

In [ ]:
import selenium
import pandas as pd
import time
import folium
import matplotlib.pyplot as plt
import seaborn as sns
import koreanize_matplotlib
import requests
import pandas as pd

from tqdm import tqdm
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.by import By
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

In [ ]:
driver = webdriver.Chrome(service=Service("../driver/chromedriver-linux64/chromedriver"))
url = "https://www.starbucks.co.kr/store/store_map.do?disp=locale"
driver.get(url)

In [ ]:
doc_link = driver.find_element(By.XPATH, '''//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/header[2]''')
doc_link.click()

In [ ]:
doc_link = driver.find_element(By.XPATH, '''//*[@id="container"]/div/form/fieldset/div/section/article[1]/article/article[2]/div[1]/div[2]/ul/li[1]/a''')
doc_link.click()

In [ ]:
time.sleep(0.5)
doc_link = driver.find_element(By.XPATH, '''//*[@id="mCSB_2_container"]/ul/li[1]/a''')
doc_link.click()

#### 서울시에 있는 스타벅스 지점 데이터 가져오기

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [ ]:
starbucks = soup.find_all(class_="quickResultLstCon")
starbucks_name = [Name.attrs["data-name"] for Name in starbucks]
starbucks_name

In [ ]:
starbucks = soup.find_all(class_="quickResultLstCon")
starbucks_address = [address.find(class_="result_details").text for address in starbucks]
starbucks_address

In [ ]:
starbucks_lat = [Lat.attrs["data-lat"] for Lat in starbucks]
starbucks_lat

In [ ]:
starbucks_long = [Long.attrs["data-long"] for Long in starbucks]
starbucks_long

In [ ]:
driver.close()

In [ ]:
print("Names length:", len(starbucks_name))
print("Addresses length:", len(starbucks_address))
print("Latitudes length:", len(starbucks_lat))
print("Longitudes length:", len(starbucks_long))

In [ ]:
starbucks_df = pd.DataFrame({
    'Name': starbucks_name,
    'Address': starbucks_address,
    'Lat': starbucks_lat,
    'Long':starbucks_long
})

starbucks_df.to_csv('../data/starbucks.csv', index=False)

starbucks_df

In [ ]:
starbucks_df['Brand'] = '스타벅스'

starbucks_df.to_csv('../data/starbucks.csv', index=False)

starbucks_df

# 이디야

In [ ]:
driver = webdriver.Chrome(service=Service("../driver/chromedriver-linux64/chromedriver"))
url = "https://ediya.com/contents/find_store.html"
driver.get(url)

In [ ]:
doc_link = driver.find_element(By.XPATH, '''//*[@id="contentWrap"]/div[3]/div/div[1]/ul/li[2]/a''')
doc_link.click()

In [ ]:
search_doc = driver.find_element(By.XPATH, '''//*[@id="keyword"]''')
doc_link.click()

#### 서울시의 이디야 데이터 가져오기

In [ ]:
html = driver.page_source
soup = BeautifulSoup(html, "html.parser")

In [ ]:
seoul_districts = ['서울 강남구', '서울 영등포구', '서울 마포구', '서울 강동구', '서울 강북구', '서울 강서구', '서울 관악구', '서울 광진구', '서울 구로구', '서울 금천구', '서울 노원구', '서울 도봉구', '서울 동대문구', '서울 동작구', '서울 서대문구', '서울 서초구', '서울 성동구', '서울 성북구', '서울 송파구', '서울 양천구', '서울 은평구', '서울 종로구', '서울 중랑구', '서울 중구', '서울 용산구']

In [ ]:
ediya_list = []

for district in tqdm(seoul_districts, desc="서울시의 이디야 탐색 중"):
    search_doc = driver.find_element(By.XPATH, '''//*[@id="keyword"]''')
    search_doc.click()
    search_doc.clear()
    search_doc.send_keys(district)
    doc_link = driver.find_element(By.XPATH, '''//*[@id="keyword_div"]/form/button''')
    doc_link.click()
    time.sleep(0.5)  
    
    ediya_list_element = driver.find_element(By.ID, "placesList")
    addresses = ediya_list_element.find_elements(By.TAG_NAME, "li")
    
    for address in addresses:
        ediya_list.append((district, address.text))

driver.quit()


ediya_df = pd.DataFrame(ediya_list, columns=['District', 'Address'])
ediya_df['Brand'] = '이디야'
print(ediya_df)

ediya_df.to_csv('../data/ediya_locations.csv', index=False)

In [ ]:
ediya_df

In [ ]:
ediya_df["Name"] = ediya_df["Address"].str.split('\n').str.get(0)
ediya_df

In [ ]:
ediya_df["Address"] = ediya_df["Address"].apply(lambda x: '\n'.join(x.split('\n')[1:]))
ediya_df

In [ ]:
ediya_df = ediya_df.drop('District', axis=1)
ediya_df

In [ ]:
new_order = ['Name', 'Address', 'Brand']

ediya_df = ediya_df[new_order]

ediya_df

##### 위도, 경도가 결측치인 데이터 수정

In [ ]:
ediya_df.loc[ediya_df['Address'] == '서울 영등포구 영등포로35길 19 (영등포동6가)', 'Address'] = '서울 영등포구 영등포로35가길 1'

ediya_df.loc[ediya_df['Address'] == '서울 마포구 백범로 100 (대흥동)', 'Address'] = '서울특별시 마포구 대흥동 263-1'


ediya_df.loc[ediya_df['Address'] == '서울 강북구 도봉로 207 -3(미아동)', 'Address'] = '서울특별시 강북구 미아동 207-3'


ediya_df.loc[ediya_df['Address'] == '서울 노원구 한글비석로 409 (상계동) 1~2층', 'Address'] = '서울 노원구 한글비석로 409'

ediya_df.loc[ediya_df['Address'] == '서울 동작구 사당로16가길 96 (사당동) 1,2층', 'Address'] = '서울특별시 동작구 사당동 224-13'

ediya_df.loc[ediya_df['Address'] == '서울 은평구 서오릉로 195 (구산동)', 'Address'] = '서울특별시 은평구 구산동 200-1'

In [ ]:
indexes_to_drop = ediya_df[ediya_df['Address'] == '서울 영등포구 영등포로35가길 1'].index

ediya_df = ediya_df.drop(indexes_to_drop)

In [ ]:
indexes_to_drop = ediya_df[ediya_df['Address'] == '서울특별시 마포구 대흥동 263-1'].index

ediya_df = ediya_df.drop(indexes_to_drop)


In [ ]:
indexes_to_drop = ediya_df[ediya_df['Address'] == '서울특별시 강북구 미아동 207-3'].index

ediya_df = ediya_df.drop(indexes_to_drop)


In [ ]:
indexes_to_drop = ediya_df[ediya_df['Address'] == '서울 노원구 한글비석로 409'].index

ediya_df = ediya_df.drop(indexes_to_drop)


In [ ]:
indexes_to_drop = ediya_df[ediya_df['Address'] == '서울특별시 동작구 사당동 224-13'].index

ediya_df = ediya_df.drop(indexes_to_drop)


In [ ]:
indexes_to_drop = ediya_df[ediya_df['Address'] == '서울특별시 은평구 구산동 200-1'].index

ediya_df = ediya_df.drop(indexes_to_drop)


##### 카카오맵을 이용하여 이디야 위도, 경도 정보 가져오기

In [ ]:
KAKAO_API_KEY = ''

latitudes = []
longitudes = []

headers = {"Authorization": f"KakaoAK {KAKAO_API_KEY}"}

for address in ediya_df['Address']:
    url = f'https://dapi.kakao.com/v2/local/search/address.json?query={address}'
    response = requests.get(url, headers=headers)
    address_info = response.json()
    
    if address_info['documents']:
        latitude = address_info['documents'][0]['y']  
        longitude = address_info['documents'][0]['x'] 
    else:
        latitude = None
        longitude = None
    
    latitudes.append(latitude)
    longitudes.append(longitude)


ediya_df['Lat'] = latitudes
ediya_df['Long'] = longitudes

ediya_df


In [ ]:
ediya_df = ediya_df[['Name', 'Address', 'Lat', 'Long', 'Brand']]
ediya_df

##### 스타벅스, 이디야 데이터 합치기

In [ ]:
cafe_df = pd.concat([ediya_df, starbucks_df], ignore_index=True)
cafe_df.to_csv('../data/cafe.csv')
cafe_df

##### 위도 또는 경도가 결측인 데이터 확인

In [ ]:
missing_data = cafe_df[cafe_df['Lat'].isnull() | cafe_df['Long'].isnull()]
print(missing_data)

In [ ]:
folium_map = folium.Map(location=[37.5665, 126.9780], zoom_start=12) 

for index, row in cafe_df.iterrows():
    if pd.notnull(row['Lat']) and pd.notnull(row['Long']):
       
        color = 'green' if row['Brand'].lower() == '스타벅스' else 'blue'
       
        folium.Marker(
            location=[row['Lat'], row['Long']],
            popup=row['Name'],
            icon=folium.Icon(color=color)
        ).add_to(folium_map)

folium_map


In [ ]:
cafe_df

In [ ]:
cafe_df['gu'] = [x.split(' ')[1] if len(x.split(' ')) > 1 else '기본값' for x in cafe_df['Address']]
cafe_df

In [ ]:
plt.figure(figsize=(12,8))
cafe_counts = cafe_df['gu'].value_counts()
cafe_counts.plot(kind='bar')
plt.title('지역별 카페 수')
plt.xlabel('지역')
plt.ylabel('카페 수')
plt.show()

In [ ]:
cafe_grouped = cafe_df.groupby(['gu', 'Brand']).size().unstack(fill_value=0)

cafe_grouped.plot(kind='bar', stacked=False) 
plt.title('지역별 카페 브랜드 분포')
plt.xlabel('지역')
plt.ylabel('카페 수')
plt.xticks(rotation=45)  
plt.legend(title='브랜드') 
plt.show()